In [18]:
!pip install -q yfinance
!pip install -q matplotlib
!pip install -q pandas_datareader


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [19]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import datetime

# MAIN

In [51]:
# Main
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2022, 1, 1)
test_period_length = '3M'
date_range = pd.date_range(start=start_date, end=end_date, freq=test_period_length)
macro_indicators = pd.DataFrame({'DATE': date_range})

# Print the last few rows of period bins MS_Diff
macro_indicators = money_supply_average_bins('3M', macro_indicators)

macro_indicators.tail()


,DATE,MS_Diff
83,2020-10-31,316.214334
84,2021-01-31,323.094948
85,2021-04-30,308.023595
86,2021-07-31,3.821557
87,2021-10-31,2.964176


# Money Supply

In [31]:
def fetch_money_supply(start_date, end_date):
    # Define the list of money supply series to fetch
    money_supply_series = ['M1SL', 'M2SL']

    # Fetch money supply data using pandas_datareader
    money_supply_data = pdr.get_data_fred(money_supply_series, start_date, end_date)

    # Calculate the YoY percentage change
    money_supply_data['M1SL_YoY'] = money_supply_data['M1SL'].pct_change(12) * 100
    money_supply_data['M2SL_YoY'] = money_supply_data['M2SL'].pct_change(12) * 100

    return money_supply_data


In [43]:
# Retrieves all MS between start and end date and returns MS differences in each period bin
def money_supply_average_bins(period_length, macro_indicators):
    money_supply_data = fetch_money_supply(start_date, end_date)
    money_supply_data_average = money_supply_data.resample(period_length).mean()
    money_supply_data_average['MS_Diff'] = money_supply_data_average['M1SL_YoY'] - money_supply_data_average['M2SL_YoY']

    macro_indicators = pd.merge(macro_indicators, money_supply_data_average['MS_Diff'], on='DATE', how='left')
    return macro_indicators

,DATE,MS_Diff
83,2020-10-31,316.214334
84,2021-01-31,323.094948
85,2021-04-30,308.023595
86,2021-07-31,3.821557
87,2021-10-31,2.964176
